In [1]:
import numpy as np
import pandas as pd
from models import Hankel,Rank,Cluster,Meepc
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.cluster import KMeans

In [2]:
hankel = Hankel()
rank = Rank()
cluster = Cluster()
meepc = Meepc.MEEPC()

In [3]:
df = pd.read_csv('~/data/training_data.csv')
df_normal=df[df['Attack LABLE (1:No Attack, -1:Attack)']==1]
df_attack=df[df['Attack LABLE (1:No Attack, -1:Attack)']==-1]
sensors = [3,9,25,43]
df_normal = df_normal.iloc[:, sensors]
df_attack=df_attack.iloc[:,sensors]

In [4]:
X = df_normal.iloc[:,2].values
X = hankel.fit(X,10,0.5)
X=X.T # size--> 1069X1800
kmeans,optimal_k = cluster.fit(X)
print(optimal_k)
radiis_normal = []
weights=[]
centers=[]
clusters_R=[]
clusters_V=[]
for i in range(optimal_k):
    cluster_ = X[np.where(kmeans.labels_ == i)[0]]
    r = rank.fit(cluster_)
    clusters_R.append(r)
    print("r: "+str(r))
    U,Sigma,VT = np.linalg.svd(cluster_)
    V = VT.T
    clusters_V.append(V)
    cluster_ = np.matmul(cluster_,V[:,:r])
    weight,center = meepc.fit(cluster_)
    weights.append(weight)
    centers.append(center)
    var1=np.square(cluster_-center)
    var2=np.matmul(weight,var1.T)
    radiis_normal.append(np.sqrt(var2))

4
r: 9
r: 11
r: 9
r: 9


In [5]:
X_attack = df_attack.iloc[:,2].values
X_att = hankel.fit(X_attack,10,0.5)
X_att=X_att.T # size--> X1800
kmeans = KMeans(n_clusters=optimal_k, init='k-means++')
kmeans.fit(X_att)
radiis_attack = []
print(X_att.shape)
for i in range(optimal_k):
    cluster_ = X_att[np.where(kmeans.labels_ == i)[0]]
    r = rank.fit(cluster_)
    print("r: "+str(r))
    U,Sigma,VT = np.linalg.svd(cluster_)
    V = VT.T
    cluster_ = np.matmul(cluster_,V[:,:r])
    weight,center = meepc.fit(cluster_)
    weights.append(weight)
    centers.append(center)
    var1=np.square(cluster_-center)
    var2=np.matmul(weight,var1.T)
    radiis_attack.append(np.sqrt(var2))

(27, 600)
r: 3
r: 9
r: 7
r: 1


In [6]:
# def calc_radius(X,kmeans,optimal_k):
#     radiis = []
#     weights = []
#     centers = []
#     topR =[ ]
#     cluster_V = []
#     for i in range(optimal_k):
#         cluster_points = X[np.where(kmeans.labels_ == i)[0]]
#         r = rank.fit(X)
#         print('r: ',r)
#         topR.append(r)
#         U,Sigma,VT = np.linalg.svd(cluster_points)
#         V = VT.T
#         cluster_V.append(V[:,:r])
#         cluster_points = np.matmul(cluster_points,V[:,:r])
#         print(cluster_points.shape)
#         weight,center = meepc.fit(cluster_points)
#         weights.append(weight)
#         centers.append(center)
#         var1=np.square(cluster_points-center)
#         var2=np.matmul(weight,var1.T)
#         # max_radius=np.amax(var2)
#         radiis.append(np.sqrt(var2))
#     return radiis,weights,centers,cluster_V,topR

In [7]:
# X_n = df_normal.iloc[:,2].values
# X_n = hankel.fit(X_n,10,0.5)
# X_n = X_n.T
# kmeans,optimal_k = cluster.fit(X_n)
# print(optimal_k)
# radiis_normal = []
# for i in range(optimal_k):
#     cluster_points = X_n[np.where(kmeans.labels_ == i)[0]]
#     r = rank.fit(X_n)
#     print('r: ',r)
#     U,Sigma,VT = np.linalg.svd(cluster_points)
#     V = VT.T
#     cluster_points = np.matmul(cluster_points,V[:,:r])
#     print(cluster_points.shape)
#     weight,center = meepc.fit(cluster_points)
#     var1=np.square(cluster_points-center)
#     var2=np.matmul(weight,var1.T)
#     radiis_normal.append(np.sqrt(var2))
# # radiis_normal,weights_normal,centers_normal,cluster_V_normal,topR_normal = calc_radius(X_n,kmeans,optimal_k)
# # radiis_normal

In [8]:
# X_a = df_attack.iloc[:,2].values
# X_a = hankel.fit(X_a,10,0.5)
# X_a = X_a.T
# kmeans = KMeans(n_clusters=optimal_k, init='k-means++')
# kmeans.fit(X_a)
# radiis_attack,weights_attack,centers_attack,cluster_V_attack,topR_attack = calc_radius(X_a,kmeans,optimal_k)
# radiis_attack

In [9]:
def find_threshold(radii_n,radii_a):
    label = [1]*len(radii_n) + [-1]*len(radii_a)
    label = np.array(label)
    radiis = np.array(list(radii_n)+list(radii_a))
    indices = np.argsort(radiis)
    label = label[indices]
    pos_cumsum = np.zeros(len(label))
    neg_cumsum = [0]*len(label)
    if label[0] > 0:
        pos_cumsum[0] = 1
    else:
        neg_cumsum[0] = 1
    for i in range(1,len(label)):
        if label[i] > 0:
            pos_cumsum[i] = 1+pos_cumsum[i-1]
            neg_cumsum[i] = neg_cumsum[i-1]
        else:
            neg_cumsum[i] = 1+neg_cumsum[i-1]
            pos_cumsum[i] = pos_cumsum[i-1]
    idx = np.argmax(pos_cumsum-neg_cumsum)
    print(indices[idx])
    return radiis[indices[idx]]

In [10]:
threshold_clusters = [0]*optimal_k
for i in range(optimal_k):
    threshold_clusters[i] = find_threshold(radiis_normal[i],radiis_attack[i])
threshold_clusters

314
1121
26
63


[0.9981574681523254,
 0.9879446059313399,
 0.9986001021045503,
 0.9960051276200057]

In [11]:
df_test=pd.read_csv('~/data/testdata.csv')
y_actual=df_test.iloc[:,-1]
df_test=df_test.iloc[:,sensors]

In [14]:
labels=hankel.fit(np.array(y_actual),10,0.5)
isatt=np.any(labels<0,axis=0)
att_indices=np.where(isatt)[0]
att_indices

array([803, 804, 805, 806, 807, 808, 809, 820, 821, 822, 823])

In [15]:
X_test=df_test.iloc[:,2].values
Xtest=hankel.fit(X_test,10,0.5)  #size -> 1800x274
Xtest=Xtest.T  #size -> 274x1800
print(Xtest.shape)
distances=[]
for i in range(optimal_k):
    Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) #size -> 274 x R
    var1=np.square(Xtest_cluster-centers[i])
    var2=np.matmul(weights[i],var1.T)
    distance=np.sqrt(var2)
    distances.append(distance)

(826, 600)


In [16]:
# distances matrix
stacked_arr=np.vstack(distances)
distances_arr=np.transpose(stacked_arr)
distances_arr #-> 274 X 4

array([[2.60805818, 0.44251985, 1.56260145, 1.0773043 ],
       [2.61646126, 0.44338015, 1.54689978, 1.09447048],
       [2.51987635, 0.36833168, 1.45386457, 1.05341729],
       ...,
       [2.12496156, 0.39945339, 1.23894861, 0.76135433],
       [2.02341986, 0.45422115, 1.07570108, 0.88248018],
       [2.02405096, 0.58633963, 1.27847725, 0.77663493]])

In [17]:
check_anamoly=threshold_clusters-distances_arr     # positive = inside cluster | negative = outside cluster
check_anamoly

array([[-1.60990071,  0.54542476, -0.56400135, -0.08129917],
       [-1.61830379,  0.54456446, -0.54829968, -0.09846536],
       [-1.52171888,  0.61961292, -0.45526447, -0.05741216],
       ...,
       [-1.12680409,  0.58849121, -0.24034851,  0.2346508 ],
       [-1.0252624 ,  0.53372346, -0.07710098,  0.11352495],
       [-1.0258935 ,  0.40160498, -0.27987714,  0.21937019]])

In [18]:
#check for if any datapoint is coming inside the cluster
is_positive=np.any(check_anamoly>0,axis=1)
#indices of those rows
positive_indices=np.where(is_positive)[0]

In [19]:
for i in range(826):
  if i not in positive_indices:
    print(i)

266
281
285
287
738


In [ ]:
# array([803, 804, 805, 806, 807, 808, 809, 820, 821, 822, 823])